# South Sudan Data Layers

This notebook is used to prepare the data layers for South Sudan. The data layers will be processed and upload to Mapbox.

## Data Hierarchy

The data is organized in the following hierarchy:

- [This](https://docs.google.com/spreadsheets/d/1RdJCjygAiWu2zBMGRF0ayigzrA2WhaObWMNdlkllgSQ/edit?usp=sharing) is the link to the data hierarchy spreadsheet.

## Data Access
**Input Data**

The data is stored in the following Google Cloud Storage bucket (source from GMV):
- https://console.cloud.google.com/storage/browser/wbhydross_deliverables

**Output Data**
- Raster layers: s3 bucket
- Vector layers: Mapbox


## Setup

### Library import


In [1]:
# imports
import os
import sys
from pathlib import Path
from pprint import pprint

# Include local library paths if you have ../src/utils.py
sys.path.append("../src/")
sys.path.append("../src/animations")
sys.path.append("../src/datasets")
sys.path.append("../src/helpers")
sys.path.append("../src/datasets/factory")

from datasets.datasets import dataset_database
from datasets.processing import LayerProcessing
from helpers.mapbox_uploader import upload_to_mapbox
from helpers.s3_uploader import upload_directory_to_s3
from helpers.settings import get_settings

In [2]:
# Load settings with environment variables
settings = get_settings()

# Data Acquisition

## Dataset information

In [3]:
datasets = dataset_database.datasets()
pprint(datasets)

{'Agricultural drought exposure': <datasets.datasets.Dataset object at 0x7f032b59a180>,
 'Agricultural drought hazard': <datasets.datasets.Dataset object at 0x7f032b1131d0>,
 'Boundaries': <datasets.datasets.Dataset object at 0x7f032b1131a0>,
 'Contextual layers': <datasets.datasets.Dataset object at 0x7f032b1130e0>,
 'EO-based flood exposure': <datasets.datasets.Dataset object at 0x7f032b113170>,
 'EO-based flood hazard': <datasets.datasets.Dataset object at 0x7f032b113110>,
 'Hydrographic data': <datasets.datasets.Dataset object at 0x7f032b113140>,
 'Hydrometeorological Data': <datasets.datasets.Dataset object at 0x7f032b113080>,
 'Meteorological drought exposure': <datasets.datasets.Dataset object at 0x7f032b1130b0>,
 'Meteorological drought hazard': <datasets.datasets.Dataset object at 0x7f032b113020>,
 'Model-based flood exposure': <datasets.datasets.Dataset object at 0x7f032b113050>,
 'Model-based flood hazard': <datasets.datasets.Dataset object at 0x7f032b112ff0>,
 'Populated in

# Static Layers
## Floods and Droughts Layers
### Create layers

In [4]:
datasets_list = [
    "Model-based flood hazard",
    "EO-based flood hazard",
    "Meteorological drought hazard",
    "Agricultural drought hazard",
    "Model-based flood exposure",
    "EO-based flood exposure",
    "Meteorological drought exposure",
    "Agricultural drought exposure",
]

dict_path = "../data/processed/datasets_dict.json"

layer_processing = LayerProcessing(datasets, datasets_list, dict_path)
layer_processing.create_layers()

  0%|          | 0/8 [00:00<?, ?it/s]

Model-based flood hazard


100%|██████████| 10/10 [00:00<00:00, 180013.05it/s]


EO-based flood hazard


100%|██████████| 2/2 [00:00<00:00, 50533.78it/s]


...

100%|██████████| 10/10 [00:00<00:00, 283398.92it/s]


Meteorological drought exposure


100%|██████████| 8/8 [00:00<00:00, 277309.36it/s]


Agricultural drought exposure


100%|██████████| 8/8 [00:00<00:00, 536.48it/s]


### Raster layers

#### Upload raster tiles to S3 bucket

In [ ]:
directory_path = Path("../data/processed/RasterTiles/")
bucket_name = "wims-ss-staging-assets-bucket"
bucket_folder = "raster-tiles"

all_folders = os.listdir(directory_path)

for folder in all_folders:
    local_directory = directory_path / Path(folder)
    if os.path.isdir(local_directory):
        upload_directory_to_s3(local_directory, bucket_name, f"{bucket_folder}/{folder}")
        print(folder)

### Vector layers

#### Upload layers to Mapbox

In [5]:
directory_path = Path("../data/processed/VectorLayers/")

all_files = os.listdir(directory_path)

for file_name in all_files:
    local_file = directory_path / Path(file_name)

    # Upload to Mapbox
    upload_name = upload_to_mapbox(
        local_file,
        file_name,
        settings.MAPBOX_USER,
        settings.MAPBOX_TOKEN,
    )

## Contextual Layers
### Create layers

In [11]:
datasets_list = ["Contextual layers"]

dict_path = "../data/processed/datasets_dict.json"

layer_processing = LayerProcessing(datasets, datasets_list, dict_path)
layer_processing.create_layers()

  0%|          | 0/1 [00:00<?, ?it/s]

Contextual layers


INFO:helpers.raster_processor:Applying styles
Application path not initialized
INFO:helpers.raster_processor:Converting to GeoTIFF


Processing Land cover from Contextual layers


INFO:helpers.raster_processor:Converting to Cloud-Optimized GeoTIFF
Reading input: /home/iker/Vizzuality/Proiektuak/wims-south-sudan/data-processing/data/processed/RasterLayers/CL_land_cover.tif

Adding overviews...
Updating dataset tags...
Writing output to: /home/iker/Vizzuality/Proiektuak/wims-south-sudan/data-processing/data/processed/RasterLayers/CL_land_cover.tif
INFO:helpers.raster_processor:Processing complete. Output saved to ../data/processed/RasterLayers/CL_land_cover.tif


...

Processing Population from Contextual layers


Application path not initialized
INFO:helpers.raster_processor:Converting to GeoTIFF
INFO:helpers.raster_processor:Converting to Cloud-Optimized GeoTIFF
Reading input: /home/iker/Vizzuality/Proiektuak/wims-south-sudan/data-processing/data/processed/RasterLayers/CL_population.tif

Adding overviews...
Updating dataset tags...
Writing output to: /home/iker/Vizzuality/Proiektuak/wims-south-sudan/data-processing/data/processed/RasterLayers/CL_population.tif
INFO:helpers.raster_processor:Processing complete. Output saved to ../data/processed/RasterLayers/CL_population.tif
Warning 1: Unable to save auxiliary information in https://storage.googleapis.com/wbhydross_deliverables/D3-Database/01-Population_Assets_Infrastructures/Population-GHS/WBHYDROSSD_GHS-POP_4326_100m_SouthSudan_2020_20240429.tif.aux.xml.


Creating tiles ...


Processing Seasonal cattle grazing areas from Contextual layers
Loading data from https://storage.googleapis.com/wbhydross_deliverables/D3-Database/01-Population_Assets_Infrastructures/Pastures-REACH/WBHYDROSSD_REACH-SeasonalCattleGrazingAreas_4326_SouthSudan_2020_20240103.shp...


INFO:root:Creating JSON file...
INFO:pyogrio._io:Created 80 records
INFO:root:Creating mbtiles file...
For layer 0, using name "CL_seasonal_cattle_grazing_areas"
../data/processed/RasterLayers/CL_seasonal_cattle_grazing_areas.json:8: Found ] at top level
../data/processed/RasterLayers/CL_seasonal_cattle_grazing_areas.json:11: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","name":"CL_seasonal_cattle_grazing_areas","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
80 features, 473626 bytes of geometry, 591 bytes of separate metadata, 1108 bytes of string pool
Choosing a maxzoom of -z1 for features about 252062 feet (76829 meters) apart
Choosing a maxzoom of -z10 for resolution of about 440 feet (134 meters) within features
  99.9%  10/612/497  
100%|██████████| 1/1 [30:06<00:00, 1806.02s/it]


## Boundaries, Waterbodies and Infrastructure 
### Create layers

In [4]:
datasets_list = [
    "Boundaries",
    "Hydrographic data",
    "Populated infrastructures",
    "Transportation Network Infrastructures",
    "Water-related infrastructures",
]

dict_path = "../data/processed/datasets_dict.json"

layer_processing = LayerProcessing(datasets, datasets_list, dict_path)
layer_processing.create_layers()

  0%|          | 0/5 [00:00<?, ?it/s]

Boundaries


100%|██████████| 5/5 [00:00<00:00, 90394.48it/s]


Hydrographic data


INFO:helpers.raster_processor:Applying styles
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized


...

100%|██████████| 2/2 [00:00<00:00, 79891.50it/s]


Transportation Network Infrastructures


100%|██████████| 1/1 [00:00<00:00, 41943.04it/s]


Water-related infrastructures


100%|██████████| 5/5 [03:37<00:00, 43.55s/it] 


# Animated Layers
## Hydrometeorological Data Layers
### Create layers

In [4]:
datasets_list = [
    "Hydrometeorological Data",
]

dict_path = "../data/processed/datasets_dict.json"

layer_processing = LayerProcessing(datasets, datasets_list, dict_path)
layer_processing.create_layers()

  0%|          | 0/1 [00:00<?, ?it/s]

Hydrometeorological Data


Processing Evapotranspiration from Hydrometeorological Data
Loading Zarr data from gs://wbhydross_deliverables/D3-Database/02- Meteorological datasets/Evapotranspiration-WaPOR/WBHYDROSSD_WaPOR_Evapotranspiration_100m_SouthSudan_2023_20240220.zarr...
Creating tiles ...
